In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

In [3]:
#u_l = ['4/', '40/', '3/', '6/', '80/', '2/']
u_l = ['2/']
url_basic = 'https://www.kommersant.ru/archive/rubric/'
res = pd.DataFrame()
t0 = time.time()

for u in u_l:
    
    url_ = url_basic + u
    
    for y in range(2018,2021):
        for m in range(1,13):
            for d in range(1,32):

                    st_m = str(m)
                    if len(st_m) == 1: st_m = '0'+st_m

                    st_d = str(d)
                    if len(st_d) == 1: st_d = '0'+st_d

                    st = str(y) + '-' + st_m + '-' + st_d

                    url = url_ + st

                    try: 
                        page = requests.get(url)
                    except:
                        r = {'step':st,'rubric':u, 'error':'downloading error', 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))
                        continue

                    try:
                        p = BeautifulSoup(page.content, "html.parser")
                        url_list = p.find_all('article')
                    except:
                        r = {'step':st,'rubric':u, 'error':'parsing error', 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))


                    for x in url_list:
                        r = {'step':st,'rubric':u, 'url':x.find('a').get('href'), 'time':(time.time()- t0)}
                        res = res.append(pd.DataFrame(r, index = [len(res)]))

                    print(st, url_, end = '\r')
        res.to_csv('kommersant_url_00.csv', sep='|')

res.to_csv('kommersant_url_00.csv', sep='|')
res.to_excel('kommersant_url_00.xlsx')
res.head()

,step,rubric,url,time
0,2018-01-01,2/,/doc/4729212?from=hotnews,0.873878
1,2018-01-01,2/,/doc/4729221?from=hotnews,0.874879
2,2018-01-01,2/,/doc/4729225?from=hotnews,0.875827
3,2018-01-01,2/,/doc/4729252,0.876824
4,2018-01-01,2/,/doc/4729249,0.877821


In [8]:
a = pd.concat([b, res], ignore_index=True)
a.shape

(857868, 5)

In [11]:
a.shape

(857868, 5)

In [ ]:
a.tail()

In [12]:
a = a.drop_duplicates(subset=['url'], keep='first')
a.shape

(79788, 5)

In [13]:
a.to_csv('kommersant_url_0_uniq.csv', sep='|')
a.to_excel('kommersant_url_0_uniq.xlsx')

In [15]:
res = a.copy()

In [16]:
def find_doc(x):
    return x.find('/doc/') != -1

res['flag'] = res['url'].apply(find_doc)
res[res['flag']]['rubric'].value_counts()

4/     20672
6/     17889
2/     15588
40/    11694
3/     11186
80/     2599
Name: rubric, dtype: int64

In [17]:
res = res[res['flag']]
res = res.drop_duplicates(['url'])
res.to_csv('kommersant_url.csv', index = False, sep='|')
url_list = res.copy()
res.head()

,Unnamed: 0,step,rubric,url,time,flag
0,0.0,2014-01-01,4/,/doc/4729200?from=hotnews,0.866174,True
1,1.0,2014-01-01,4/,/doc/4729195?from=hotnews,0.867172,True
2,2.0,2014-01-01,4/,/doc/4729198?from=hotnews,0.868169,True
3,3.0,2014-01-01,4/,/doc/4729207,0.869166,True
4,4.0,2014-01-01,4/,/doc/4729206,0.870164,True


In [44]:
urls = url_list['url'].tolist()
len(urls)

79628

In [45]:
url_ = 'https://www.kommersant.ru'
res = pd.DataFrame()

#urls = url_list['url'].tolist()
urls = urls[33766:]


for x_ in tqdm(range(len(urls)), position=0, leave=True):
    
    x = urls[x_]
    url = url_ + x

    try:
        page = requests.get(url).content
    except:
        r = {'url': x, "error": "downloading error"}
        res = res.append(pd.DataFrame(r, index=[len(res)]))
        continue

    p = BeautifulSoup(page, "html.parser")
    p_ = p.find_all('p', {'class': "b-article__text"})
    st = ''
    for t in p_: st = st + t.text + ' '

    try:
        article_name = p.find_all('h2', {'class': "article_name"})[0].text
    except:
        try:
            article_name = p.find_all('h1', {'class': "article_name"})[0].text
        except:
            article_name = None

    try:
        article_subheader = p.find_all('h1', {'class': "article_subheader"})[0].text
    except:
        article_subheader = None

    try:
        authors = p.find_all('p', {'class': "b-article__text document_authors"})[0].text
    except:
        authors = None
    
    try:
        dt = str(p.find_all('meta', {'property':'article:published_time'})).split('"')[1]
    
        dt2 = str(p.find_all('meta', {'itemprop':'datePublished'})).split('"')[1]
    except:
        dt = None
        dt2 = None
    r = {'url': url, "text": st, "authors": authors, "article_header": article_name,
         "artical_subheader": article_subheader, "date": dt, "date2": dt2}
    res = res.append(pd.DataFrame(r, index=[len(res)]))
    if x_ % 100 == 0 or x_ in (20671, 17889, 15588, 11694, 11186, 2599):
        res.to_csv('kommersant_final.csv', index = False, sep='|')
    
res.to_csv('kommersant_final.csv', index = False, sep='|')
res.to_excel('kommersant_final.xlsx')
res.head()

100%|██████████████████████████████████████████████████████████████████████████| 45862/45862 [5:03:24<00:00,  2.52it/s]


,url,text,authors,article_header,artical_subheader,date,date2
0,https://www.kommersant.ru/doc/2660588,Доля зеленой энергетики в РФ может составить д...,Ангелина Давыдова,Кризису не остановить ветряки,Перспективы зеленой энергетики в России ухудша...,2015-02-04T18:10:10+03:00,2015-02-04T18:10:10+03:00
1,https://www.kommersant.ru/doc/2660538,,None,Какую пенсию вам насчитают,None,None,None
2,https://www.kommersant.ru/doc/2660211,Банк России в мартовском прогнозе может снизит...,Дмитрий утрин,Инфляция разорится первой,"Банк России предполагает, что рост цен будет о...",2015-02-04T00:00:00+03:00,2015-02-04T00:00:00+03:00
3,https://www.kommersant.ru/doc/2660208,Вчера на совещании у Владимира Путина в Ново-О...,Вадим Вислогузов,Фонд перераспределенного достояния,Владимир Путин утвердил список отобранных к фи...,2015-02-04T00:00:00+03:00,2015-02-04T00:00:00+03:00
4,https://www.kommersant.ru/doc/2660207,Через год после начала работы нового законодат...,"Анастасия Мануйлова, Дмитрий Бутрин",Ведомства строго отсчитали,Счетная палата имеет претензии к контрактной с...,2015-02-04T00:00:00+03:00,2015-02-04T00:00:00+03:00


In [46]:
res1 = pd.read_csv('kommersant_final0.csv', sep='|')


,url,text,authors,article_header,artical_subheader,date,date2
0,https://www.kommersant.ru/doc/2660588,Доля зеленой энергетики в РФ может составить д...,Ангелина Давыдова,Кризису не остановить ветряки,Перспективы зеленой энергетики в России ухудша...,2015-02-04T18:10:10+03:00,2015-02-04T18:10:10+03:00
1,https://www.kommersant.ru/doc/2660538,,None,Какую пенсию вам насчитают,None,None,None
2,https://www.kommersant.ru/doc/2660211,Банк России в мартовском прогнозе может снизит...,Дмитрий утрин,Инфляция разорится первой,"Банк России предполагает, что рост цен будет о...",2015-02-04T00:00:00+03:00,2015-02-04T00:00:00+03:00
3,https://www.kommersant.ru/doc/2660208,Вчера на совещании у Владимира Путина в Ново-О...,Вадим Вислогузов,Фонд перераспределенного достояния,Владимир Путин утвердил список отобранных к фи...,2015-02-04T00:00:00+03:00,2015-02-04T00:00:00+03:00
4,https://www.kommersant.ru/doc/2660207,Через год после начала работы нового законодат...,"Анастасия Мануйлова, Дмитрий Бутрин",Ведомства строго отсчитали,Счетная палата имеет претензии к контрактной с...,2015-02-04T00:00:00+03:00,2015-02-04T00:00:00+03:00


In [48]:
res1['date2'] = res1['date']
res1.head()

,url,text,authors,article_header,artical_subheader,date,date2
0,https://www.kommersant.ru/doc/4729200?from=hot...,Из отделов полиции в Москве отпустили всех зад...,NaN,Всех задержанных на форуме муниципальных депут...,NaN,2021-03-13T23:34:06+03:00,2021-03-13T23:34:06+03:00
1,https://www.kommersant.ru/doc/4729195?from=hot...,Сенатор от Тюменской области заместитель предс...,NaN,Умер зампред экономического комитета Совета фе...,NaN,2021-03-13T21:34:08+03:00,2021-03-13T21:34:08+03:00
2,https://www.kommersant.ru/doc/4729198?from=hot...,Исследования вакцины от коронавируса «Спутник ...,NaN,Гинцбург: в России начались испытания «Спутник...,NaN,2021-03-13T22:15:38+03:00,2021-03-13T22:15:38+03:00
3,https://www.kommersant.ru/doc/4729207,В немецком Дрездене прошла демонстрация против...,NaN,В Дрездене 12 полицейских пострадали в ходе ак...,NaN,2021-03-14T02:47:01+03:00,2021-03-14T02:47:01+03:00
4,https://www.kommersant.ru/doc/4729206,Опреснительная станция в Ялте обойдется в 3 мл...,NaN,Стоимость опреснительной установки в Ялте оцен...,NaN,2021-03-14T02:24:40+03:00,2021-03-14T02:24:40+03:00


In [49]:
res1.shape

(33701, 7)

In [50]:
res.shape

(45862, 7)

In [52]:
fin = pd.concat([res1, res])
fin.shape

(79563, 7)

In [57]:
fin.columns

Index(['url', 'text', 'authors', 'article_header', 'artical_subheader', 'date',
       'date2'],
      dtype='object')

In [60]:
fin.shape

(79563, 4)

In [59]:
fin.to_csv('kommersant_final.csv', index = False, sep='|')
fin.to_excel('kommersant_final.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/2607567' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/2607543' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/2607505' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/2607514' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicod

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/2890888' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/2890849' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/2890821' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/2894730' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicod

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/3201572' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/3201570' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/3201567' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:945: UserWarning: Ignoring URL 'https://www.kommersant.ru/doc/3201542' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicod

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

